Some experiments

In [ ]:
%pip install sentence_transformers torch ipywidgets

In [1]:
import torch.nn.functional as F
import json
import os
import torch
import pickle

from bs4 import BeautifulSoup


from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [5]:
class EmbedTest:
    def __init__(self) -> None:
        self._tokenizer = AutoTokenizer.from_pretrained('./data/original')
        self._model = AutoModel.from_pretrained('./data/original')
    def clean(self):
        del self._model
        del self._tokenizer
        torch.cuda.empty_cache()
        # torch.cuda.synchronize()
        # torch.empty()
        # self.__init__(self)
    def _average_pool(self,last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0) # nonmasked -> 0
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None] # average of masked, as attention_mask is 1 or 0
    def generateEmbeddings(self,texts):
        batch_dict = self._tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')      
        outputs = self._model(**batch_dict)
        embeddings = self._average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        # normalize embeddings
        embeddings = F.normalize(embeddings, p=2, dim=1)
        del batch_dict
        del outputs
        return embeddings


Calculate article embeddings

In [6]:

def readArticle(embedder,id):
     with open(f"./data/articles/{id}") as f:
         article = json.load(f)
         text = BeautifulSoup(article["body"]).get_text()
         [embedding] = embedder.generateEmbeddings([f"passage:{text}"])
         return (article["id"],article["name"],embedding)

artfiles = os.listdir("./data/articles/")
articles = []

def readArticles(files):
    embedder = EmbedTest()
    for article in files:
        articles.append(readArticle(embedder,article))
    embedder.clean()
    del embedder

readArticles(artfiles[0:10])
print("new batch")
readArticles(artfiles[11:20])
print("new batch")
readArticles(artfiles[21:30])
# with open("./data/trained/articles.dat","wb") as f:
#     pickle.dump(articles,f)
len(artfiles)

new batch
new batch


777

read articles and embeddings

In [ ]:
with open("./data/trained/articles.dat","rb") as f:
    articles = pickle.load(f)

In [3]:
embedder = EmbedTest()
def lookup(q):
    [queryembed] = embedder.generateEmbeddings(f"query:{q}")
    def score(art):
         return (((art[2] @ queryembed) * 100).item(),art)
    scored = list(map(score, articles))
    scored.sort(reverse=True)
    def extract(art):
         return (art[0],art[1][0],art[1][1],f"https://support.basistechnologies.com/hc/en-us/articles/{art[1][0]}")
    return list(map(extract,scored[0:5]))

def test(query):
     result = lookup(query)
     for r in result:
          print(r)
     print()

# print(lookup("task"))

# print(lookup("short dump"))
# print(lookup("test impact"))

test("import errors")
test("unexpected result")
test("task")
test("short dump")

NameError: name 'articles' is not defined

In [10]:
import gc
gc.collect()
torch.cuda.empty_cache()
gc.isenabled()

True